<a href="https://colab.research.google.com/github/brainmentorspvtltd/IGDTU_ImageProcessingTF/blob/main/IG_SkinCancerImageAug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

os.environ["KAGGLE_USERNAME"] = "ravikanttyagi"
os.environ["KAGGLE_KEY"] = "095a21c789eb4728fde2b29230033273"

!kaggle datasets download nodoubttome/skin-cancer9-classesisic

100% 784M/786M [00:37<00:00, 23.7MB/s]
100% 786M/786M [00:37<00:00, 21.7MB/s]


In [ ]:
from IPython.display import clear_output

!unzip skin-cancer9-classesisic.zip

clear_output()

In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten
from tensorflow.keras.layers import LeakyReLU, Dropout, BatchNormalization
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_path = "dataset/Train"
test_path = "dataset/Test"

In [ ]:
class_names = os.listdir(train_path)

In [ ]:
class_names

['melanoma',
 'seborrheic keratosis',
 'dermatofibroma',
 'squamous cell carcinoma',
 'nevus',
 'vascular lesion',
 'actinic keratosis',
 'basal cell carcinoma',
 'pigmented benign keratosis']

In [ ]:
for name in class_names:
  path = train_path + "/" + name
  n_images = len(os.listdir(path))
  print(name, " : ", n_images)

melanoma  :  438
seborrheic keratosis  :  77
dermatofibroma  :  95
squamous cell carcinoma  :  181
nevus  :  357
vascular lesion  :  139
actinic keratosis  :  114
basal cell carcinoma  :  376
pigmented benign keratosis  :  462


In [ ]:
for name in class_names:
  path = test_path + "/" + name
  n_images = len(os.listdir(path))
  print(name, " : ", n_images)

melanoma  :  16
seborrheic keratosis  :  3
dermatofibroma  :  16
squamous cell carcinoma  :  16
nevus  :  16
vascular lesion  :  3
actinic keratosis  :  16
basal cell carcinoma  :  16
pigmented benign keratosis  :  16


In [ ]:
# Pre-processing of images
image_data_gen = ImageDataGenerator(horizontal_flip=True,
                                    shear_range=0.2,
                                    zoom_range=0.2,
                                    rescale=1/255.,
                                    rotation_range=30,
                                    vertical_flip=True
                                    )

In [ ]:
X_train = image_data_gen.flow_from_directory(train_path, target_size=(224,224), batch_size=4, class_mode="sparse")

Found 2239 images belonging to 9 classes.


In [ ]:
X_val = image_data_gen.flow_from_directory(test_path, target_size=(224,224), batch_size=4, class_mode="sparse")

Found 118 images belonging to 9 classes.


In [ ]:
model = Sequential()
model.add(Conv2D(128, (3,3), input_shape=(224,224,3), activation="relu"))
model.add(Conv2D(128, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(Conv2D(64, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(Conv2D(32, (3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dense(1024, activation=LeakyReLU(0.1)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(512, activation=LeakyReLU(0.1)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(256, activation=LeakyReLU(0.1)))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(Dense(128, activation=LeakyReLU(0.1)))
model.add(Dense(9, activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_36 (Conv2D)          (None, 222, 222, 128)     3584      
                                                                 
 conv2d_37 (Conv2D)          (None, 220, 220, 128)     147584    
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 110, 110, 128)    0         
 g2D)                                                            
                                                                 
 conv2d_38 (Conv2D)          (None, 108, 108, 64)      73792     
                                                                 
 conv2d_39 (Conv2D)          (None, 106, 106, 64)      36928     
                                                                 
 max_pooling2d_19 (MaxPoolin  (None, 53, 53, 64)       0         
 g2D)                                                 

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['acc'])

history = model.fit(X_train, epochs=10, validation_data=X_val)

Epoch 1/10
560/560 [==============================] - 200s 152ms/step - loss: 2.2967 - acc: 0.2028 - val_loss: 2.3411 - val_acc: 0.1186
Epoch 2/10
560/560 [==============================] - 82s 146ms/step - loss: 2.1164 - acc: 0.2197 - val_loss: 2.1463 - val_acc: 0.2458
Epoch 3/10
560/560 [==============================] - 81s 145ms/step - loss: 2.0245 - acc: 0.2389 - val_loss: 3.2025 - val_acc: 0.1441
Epoch 4/10
560/560 [==============================] - 82s 147ms/step - loss: 1.9753 - acc: 0.2653 - val_loss: 3.1795 - val_acc: 0.0424
Epoch 5/10
560/560 [==============================] - 92s 165ms/step - loss: 1.8470 - acc: 0.3193 - val_loss: 2.5240 - val_acc: 0.0847
Epoch 6/10
560/560 [==============================] - 83s 148ms/step - loss: 1.8465 - acc: 0.3024 - val_loss: 4.7716 - val_acc: 0.1441
Epoch 7/10
560/560 [==============================] - 83s 147ms/step - loss: 1.8017 - acc: 0.3421 - val_loss: 2.5522 - val_acc: 0.1610
Epoch 8/10
560/560 [==============================] - 